In [1]:
import os

resume_folder = "resumes"
files = os.listdir(resume_folder)

if files:
    print("✅ Found resumes:")
    for f in files:
        print(f"- {f}")
else:
    print("❌ No resumes found. Please check your uploads.")


✅ Found resumes:
- 2 new.pdf
- 1 new.pdf
- 3 new.pdf


In [2]:
import os
import PyPDF2

def extract_text_from_resumes(resume_folder):
    resume_texts = []
    for filename in os.listdir(resume_folder):
        if filename.endswith(".pdf"):
            filepath = os.path.join(resume_folder, filename)
            with open(filepath, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                resume_texts.append((filename, text))
    return resume_texts

resume_texts = extract_text_from_resumes("resumes")

if resume_texts:
    print(f"✅ Extracted text from {len(resume_texts)} resumes.")
else:
    print("❌ No text extracted. Please check if PDFs are valid.")


✅ Extracted text from 3 resumes.


In [3]:
!pip install faiss-cpu sentence-transformers


In [4]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# ✅ Load a local embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ Convert resumes into vectors
resume_embeddings = []
resume_filenames = []

for filename, text in resume_texts:
    vector = embedding_model.encode(text)  # Convert text to embedding
    resume_embeddings.append(vector)
    resume_filenames.append(filename)

# Convert to NumPy array
resume_embeddings = np.array(resume_embeddings)

print(f"✅ Generated {resume_embeddings.shape[0]} embeddings of size {resume_embeddings.shape[1]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Generated 3 embeddings of size 384


In [5]:
# ✅ Initialize FAISS Index
dimension = resume_embeddings.shape[1]  # Get embedding size
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance index
faiss_index.add(resume_embeddings)  # Add resume embeddings to index

print(f"✅ Stored {faiss_index.ntotal} resumes in FAISS")


✅ Stored 3 resumes in FAISS


In [6]:
def search_best_resume(job_description, top_k=3):
    """Find the top K best matching resumes for a given job description"""
    job_vector = embedding_model.encode(job_description)  # Convert job desc to vector
    job_vector = np.array([job_vector])  # Reshape for FAISS

    # 🔍 Search FAISS for the closest resumes
    distances, indices = faiss_index.search(job_vector, top_k)

    print("\n🔹 Top Matching Resumes:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {resume_filenames[idx]} (Similarity Score: {distances[0][i]:.2f})")

# Example Job Description
job_desc = "Looking for a data scientist with experience in Python, machine learning, and NLP."
search_best_resume(job_desc)



🔹 Top Matching Resumes:
1. 1 new.pdf (Similarity Score: 1.20)
2. 2 new.pdf (Similarity Score: 1.33)
3. 3 new.pdf (Similarity Score: 1.63)


In [7]:
!pip install gradio faiss-cpu sentence-transformers


In [8]:
import gradio as gr

# ✅ Define the chatbot function
def resume_chatbot(job_description):
    job_vector = embedding_model.encode(job_description)  # Convert job desc to vector
    job_vector = np.array([job_vector])  # Reshape for FAISS

    # 🔍 Search FAISS for the closest resumes
    distances, indices = faiss_index.search(job_vector, 3)

    # 📜 Format the response
    response = "🔹 **Top Matching Resumes:**\n\n"
    for i, idx in enumerate(indices[0]):
        response += f"✅ **{resume_filenames[idx]}** (Similarity Score: {distances[0][i]:.2f})\n"

    return response

# ✅ Create a chatbot interface using Gradio
iface = gr.Interface(
    fn=resume_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter job description..."),
    outputs="text",
    title="📄 AI Resume Screener",
    description="🔎 Enter a job description, and the chatbot will find the best matching resumes!",
)

# ✅ Launch the chatbot
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd1a512a7a1ed1ecf9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
